# Setup

In [5]:
import pandas as pd
import json

# Transformers
from transformers import AutoTokenizer
from tqdm.auto import tqdm

In [9]:
# Set API Keys
from kaggle_secrets import UserSecretsClient # API Loggins
user_secrets = UserSecretsClient()

## Hugging Face
Hugging_Face_token = user_secrets.get_secret("Hugging_Face_token")

# Login to Hugging Face
from huggingface_hub import login

login(Hugging_Face_token)

# Load Data

In [2]:
#df_HoC_2000s_raw = pd.read_csv('H:/MA_Thesis/data/Rauh_Schwalbach_2020_ParlSpeech/df_HoC_2000s.csv')

df_HoC_2000s_raw = pd.read_csv('/kaggle/input/parlspeech-2000s/df_HoC_2000s.csv')

In [4]:
df_HoC_2000s = df_HoC_2000s_raw[['date', 'agenda', 'speechnumber', 'speaker', 'party','text']]
df_HoC_2000s.columns

Index(['date', 'agenda', 'speechnumber', 'speaker', 'party', 'text'], dtype='object')

# Preprocess Data

In [6]:
# Constants
HISTORY_MAX_TOKENS = 3000  # Adjust as needed
CONVO_MIN_TOKENS = 100  # Adjust as needed
CHAT_OWNER = "Boris Johnson"

In [12]:
# Create the tokenizer to measure the length of the text
base_model_id = "meta-llama/Llama-3.2-3B"
encoder = AutoTokenizer.from_pretrained(base_model_id, 
                                        add_bos_token=False,
                                        trust_remote_code=True, 
                                        use_fast=True, force_download=False)